## Assignment 2
### Zhengjie Deng a1865926

#### Dependency version:
- Python: 3.8.16
- pandas: 1.4.2
- sklearn: 1.0.2
- nltk: 3.7
- tqdm: 4.64.1
- matplotlib: 3.7.0
- spacy: 3.5.0
- numpy: 1.23.5
- gensim: 3.8.3

In [2]:
import warnings
import gensim.downloader as api
from collections import Counter
import spacy
import pandas as pd
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords


import numpy as np
import json
import re
import os

# Install faiss
!pip install faiss
!pip install pyserini

# Download stopwords
nltk.download('stopwords')
nltk.download('wordnet2022')

# Download en_core_web_lg model
spacy.cli.download("en_core_web_sm")


# Download the pre-trained GloVe model
glove_model = api.load('glove-wiki-gigaword-100')

# ignore the warning
warnings.filterwarnings("ignore")

# Some Kaggle-wordnet patch
! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet 

# Install java for pyserini

!wget "https://download.java.net/java/GA/jdk11/9/GPL/openjdk-11.0.2_linux-x64_bin.tar.gz"
!tar -xvf openjdk-11.0.2_linux-x64_bin.tar.gz

!export JAVA_HOME='/kaggle/working/jdk-11.0.2/'
!export PATH='/kaggle/working/jdk-11.0.2/bin':$PATH
!mkdir -p /kaggle/working/jdk-11.0.2/jre/lib/amd64/server/

!ln -s /kaggle/working/jdk-11.0.2/lib/server/libjvm.so /kaggle/working/jdk-11.0.2/jre/lib/amd64/server/libjvm.so
os.environ["JAVA_HOME"] = "/kaggle/working/jdk-11.0.2/"

  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Attempting uninstall: pybind11
    Found existing installation: pybind11 2.10.4
ERROR: Cannot uninstall pybind11 2.10.4, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps pybind11==2.10.4'.
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet2022 to /usr/share/nltk_data...
[nltk_data]   Package wordnet2022 is already up-to-date!
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.2 MB/s eta 0:00:00


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
--2023-04-05 06:03:29--  https://download.java.net/java/GA/jdk11/9/GPL/openjdk-11.0.2_linux-x64_bin.tar.gz
Resolving download.java.net (download.java.net)... 23.192.208.88
Connecting to download.java.net (download.java.net)|23.192.208.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187513052 (179M) [application/x-gzip]
Saving to: ‘openjdk-11.0.2_linux-x64_bin.tar.gz.1’

openjdk-11.0.2_linu 100%[===================>] 178.83M   243MB/s    in 0.7s    

2023-04-05 06:03:30 (243 MB/s) - ‘openjdk-11.0.2_linux-x64_bin.tar.gz.1’ saved [187513052/187513052]

jdk-11.0.2/bin/jaotc
jdk-11.0.2/bin/jar
jdk-11.0.2/bin/jarsigner
jdk-11.0.2/bin/java
jdk-11.0.2/bin/javac
jdk-11.0.2/bin/javadoc
jdk-11.0.2/bin/javap
jdk-11.0.2/bin/jcmd
jdk-11.0.2/bin/jconsole
jdk-11.0.2/bin/jdb
jdk-11.0.2/bin/jdeprscan
jdk-11.0.2/bin/jdeps
jdk-11.0.2/bin/jhsdb
jdk-11.0.2/bin/jimage
jdk-11.0.2/b

### 1. Reading dataset and pre-processing

#### 1.1 Read dataset

In [3]:
# load the dataset
f_metadata_path = "../input/CORD-19-research-challenge/metadata.csv"

# construct the data frame of metadata
df = pd.read_csv(f_metadata_path)
df.head(5)

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


#### 1.2 Dropping useless columns

In [4]:
# drop useless columns for this assignment
df_trimed = df.drop(["sha", "source_x", "doi", "license", "authors", "journal", "pmc_json_files",
                    "pmcid", "pubmed_id", "mag_id", "who_covidence_id", "arxiv_id", "url", "s2_id"], axis=1)
df_trimed.head(5)


,cord_uid,title,abstract,publish_time,pdf_json_files
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...,2001-07-04,document_parses/pdf_json/d1aafb70c066a2068b027...
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...,2000-08-15,document_parses/pdf_json/6b0567729c2143a66d737...
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...,2000-08-25,document_parses/pdf_json/06ced00a5fc04215949aa...
3,2b73a28n,Role of endothelin-1 in lung disease,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,document_parses/pdf_json/348055649b6b8cf2b9a37...
4,9785vg6d,Gene expression in epithelial cells in respons...,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,document_parses/pdf_json/5f48792a5fa08bed9f560...


#### 1.3 Sampling

Given that COVID-19 was first reported in December 2019, we can limit our data selection to articles published after this date. Therefore, we will sample data from December 1st, 2019, onwards.

In [5]:
# only keep the documents published after 2019-11
df_trimed = df_trimed[df_trimed["publish_time"] > '2019-12']
df_trimed.shape[0]


958037

Check the missing values in the dataset.

In [6]:
# check the number of missing value in each column
missing_values_count = df_trimed.isnull().sum()
missing_values_count


cord_uid               0
title                488
abstract          220150
publish_time           0
pdf_json_files    628520
dtype: int64

Many rows of data are missing titles, abstracts, and PDF files, as demonstrated above. We will drop these rows from our analysis.

In [7]:
df_trimed = df_trimed.dropna(subset=['pdf_json_files', 'title', 'abstract'])
df_trimed.shape[0]


287065

Next, we will verify the validity of the PDF file URLs. Any rows with invalid URLs will be dropped from the dataset.

In [8]:
# remove all the rows that do not have the real file
import os
# tqdm progress apply
tqdm.pandas(desc="removing all the rows that do not have the real file...")
print(df_trimed['pdf_json_files'])


df_trimed = df_trimed[df_trimed['pdf_json_files'].progress_apply(
    lambda x: os.path.isfile("../input/CORD-19-research-challenge/"+x))]
df_trimed.shape[0]


4160       document_parses/pdf_json/d84f1c9036052ca0d9f82...
4174       document_parses/pdf_json/aa3858e3cf21baa00c6f6...
4177       document_parses/pdf_json/23d0641f1f78528bbf3fa...
4179       document_parses/pdf_json/9be8c80283a7fed59057d...
4180       document_parses/pdf_json/d5d29d1d48e1a4fe8f39d...
                                 ...                        
1056653    document_parses/pdf_json/96fd9bcc269abf4e0ef2c...
1056655    document_parses/pdf_json/d86cfba423427426c10fa...
1056657    document_parses/pdf_json/0d1698212a1fcbbed2fff...
1056658    document_parses/pdf_json/9b2bbb5ceb21084931b5d...
1056659    document_parses/pdf_json/c7ae1a19b70e6e2806b67...
Name: pdf_json_files, Length: 287065, dtype: object


removing all the rows that do not have the real file...: 100%|██████████| 287065/287065 [15:50<00:00, 301.98it/s]


NameError: name 's' is not defined

Based on the keywords of the test queries, we can sample the data further. We will only keep the rows whose title or abstract contain the keywords in the test queries. 

In [9]:
# function: select the rows whose title or abstract contain the given strings, ignoring the case
def select_rows_contain_string(df, strings):
    tqdm.pandas(
        desc="selecting the rows whose title or abstract contain the given strings...")
    return df[df.progress_apply(lambda row: any(string in row['title'].lower() or string in row['abstract'].lower() for string in strings), axis=1)]

In [10]:
# title or abstract contain "COVID-19", "SARS-CoV-2", "coronavirus", "2019-nCoV", "covid", "covid-19", "Covid-19"
strings = ["COVID-19", "SARS-CoV-2", "coronavirus",
           "2019-nCoV", "covid", "covid-19", "Covid-19"]
df_trimed_covid = select_rows_contain_string(df_trimed, strings)

keyword_sampled_df_list = []

# title or abstract contain "origin", "Wuhan" from the df_trimed_covid
strings = ["origin", "Wuhan"]
df_trimed_origin = select_rows_contain_string(df_trimed_covid, strings)
keyword_sampled_df_list.append(df_trimed_origin)

# title or abstract contain "rapid testing"
strings = ["rapid testing"]
df_trimed_testing = select_rows_contain_string(df_trimed_covid, strings)
keyword_sampled_df_list.append(df_trimed_testing)

# title or abstract contain "social", "distancing", "lockdown", "quarantine"
strings = ["social distancing", "lockdown", "quarantine"]
df_trimed_social = select_rows_contain_string(df_trimed_covid, strings)
keyword_sampled_df_list.append(df_trimed_social)

# title or abstract contain "transmission route"
strings = ["transmission route"]
df_trimed_transmission = select_rows_contain_string(df_trimed_covid, strings)
keyword_sampled_df_list.append(df_trimed_transmission)

# title or abstract contain "best masks", "preventing infection", "prevent infection", "preventing transmission", "prevent transmission", "preventing spread", "prevent spread"
strings = ["best masks", "preventing infection", "prevent infection",
           "preventing transmission", "prevent transmission", "preventing spread", "prevent spread"]
df_trimed_masks = select_rows_contain_string(df_trimed_covid, strings)
keyword_sampled_df_list.append(df_trimed_masks)

# title or abstract contain "hand sanitizer"
strings = ["hand sanitizer"]
df_trimed_sanitizer = select_rows_contain_string(df_trimed_covid, strings)
keyword_sampled_df_list.append(df_trimed_sanitizer)

# title or abstract contain "vaccine", "vaccination", "vaccines", "vaccinations"
strings = ["vaccine", "vaccination", "vaccines", "vaccinations"]
df_trimed_vaccine = select_rows_contain_string(df_trimed_covid, strings)
keyword_sampled_df_list.append(df_trimed_vaccine)

# title or abstract contain "Vitamin"
strings = ["Vitamin"]
df_trimed_vitamin = select_rows_contain_string(df_trimed_covid, strings)
keyword_sampled_df_list.append(df_trimed_vitamin)

# title or abstract contain "live outside the body"
strings = ["live outside the body"]
df_trimed_outside = select_rows_contain_string(df_trimed_covid, strings)
keyword_sampled_df_list.append(df_trimed_outside)

# title or abstract contain "initial symptoms"
strings = ["initial symptoms"]
df_trimed_symptoms = select_rows_contain_string(df_trimed_covid, strings)
keyword_sampled_df_list.append(df_trimed_symptoms)


selecting the rows whose title or abstract contain the given strings...: 100%|██████████| 267675/267675 [00:21<00:00, 12240.41it/s]
selecting the rows whose title or abstract contain the given strings...: 100%|██████████| 158540/158540 [00:06<00:00, 25778.83it/s]
selecting the rows whose title or abstract contain the given strings...: 100%|██████████| 158540/158540 [00:04<00:00, 39509.09it/s]
selecting the rows whose title or abstract contain the given strings...: 100%|██████████| 158540/158540 [00:08<00:00, 19152.38it/s]
selecting the rows whose title or abstract contain the given strings...: 100%|██████████| 158540/158540 [00:03<00:00, 41083.76it/s]
selecting the rows whose title or abstract contain the given strings...: 100%|██████████| 158540/158540 [00:17<00:00, 8973.72it/s]
selecting the rows whose title or abstract contain the given strings...: 100%|██████████| 158540/158540 [00:03<00:00, 42333.42it/s]
selecting the rows whose title or abstract contain the given strings...: 100%

In [11]:
# join all the dataframes above into one dataframe and remove the duplicates
df_trimed_covid = pd.concat(keyword_sampled_df_list).drop_duplicates()
df_trimed_covid.shape[0]

43311

Finally, we randomly sample 10000 rows from the dataset.

In [12]:
# randomly pick 10000 documents from the dataset

df_sampled = df_trimed_covid.sample(n=10000, random_state=42)
df_sampled

,cord_uid,title,abstract,publish_time,pdf_json_files
753376,0ex7dy6g,In this together: Psychological wellbeing of f...,The COVID‐19 pandemic and the associated infec...,2021-06-14,document_parses/pdf_json/f6cc631cda193e16e84b3...
913530,rmvw3ko0,The Impact of COVID-19 on Self-Reported Substa...,"As the COVID-19 pandemic sweeps the globe, man...",2022-04-11,document_parses/pdf_json/761ee085a77ce60863fe9...
712304,kf7lix7f,COVID-19 Pandemic and the Convalescent Plasma ...,PURPOSE OF REVIEW: COVID-19 pandemic has been ...,2021-07-03,document_parses/pdf_json/60c6c463404554c0ad087...
736570,e0eki2rk,{fr}Who is hesitant about Covid-19 vaccines ? ...,OBJECTIVES: : To report the characteristics of...,2022-03-28,document_parses/pdf_json/8ee6271860745bf4d23d6...
816108,pyxmblll,Immediate reactions following the first dose o...,Addressing COVID19 vaccine hesitancy and minim...,2021-05-14,document_parses/pdf_json/2aa4576b561138b59ca67...
...,...,...,...,...,...
768690,3mwqk02l,Administrative Governance and Frontline Office...,The spread of COVID-19 put prisons across the ...,2021-03-12,document_parses/pdf_json/13c03c9ab9c2c4a5d8356...
737001,fniymnmf,Reverse genetic systems: Rational design of co...,"Since the end of 2019, the global COVID-19 out...",2020-06-30,document_parses/pdf_json/8f2dd926eed4506073c2f...
1004462,ztrmpvbr,The cost-effectiveness of social distancing me...,BACKGROUND: As one of the strategies to mitiga...,2021-12-23,document_parses/pdf_json/febd50367c452b4be9f5b...
79013,zslbp3f4,SAPV-Patienten in der COVID-19-Krise: Eine int...,"BACKGROUND: In the outpatient setting, COVID-1...",2022-05-25,document_parses/pdf_json/8ef48ebbd9af1a8b0ffad...


#### 1.4 Pre-processing the text data 

After sampling the data, our next step is to pre-process the text data. This involves extracting the text from the PDF JSON files as our first task.

In [13]:
# function: get the pdf json text based on the URL
def extract_pdf(row, pdf_json):
    # paragraphs other than abstract, introduction, conclusion
    row["other_paragraph"] = []
    for body_paragraph in pdf_json['body_text']:
        if body_paragraph["section"] == "Introduction":
            row["introduction"] = body_paragraph["text"]
        elif body_paragraph["section"] == "Conclusion":
            row["conclusion"] = body_paragraph["text"]
        else:
            row["other_paragraph"].append(body_paragraph["text"])
    return row

# extract the pdf json text from the sampled dataset
tqdm.pandas(desc="extracting the pdf json text from the sampled dataset...")
df_sampled = df_sampled.progress_apply(lambda row: extract_pdf(
    row, json.load(open("../input/CORD-19-research-challenge/"+row['pdf_json_files']))), axis=1)
df_sampled.head()


extracting the pdf json text from the sampled dataset...: 100%|██████████| 10000/10000 [01:57<00:00, 85.15it/s]


,abstract,conclusion,cord_uid,introduction,other_paragraph,pdf_json_files,publish_time,title
753376,The COVID‐19 pandemic and the associated infec...,NaN,0ex7dy6g,NaN,"[A severe acute respiratory syndrome, coronavi...",document_parses/pdf_json/f6cc631cda193e16e84b3...,2021-06-14,In this together: Psychological wellbeing of f...
913530,"As the COVID-19 pandemic sweeps the globe, man...",NaN,rmvw3ko0,NaN,"[Since March 2020, the SARS-CoV-2 (COVID-19) p...",document_parses/pdf_json/761ee085a77ce60863fe9...,2022-04-11,The Impact of COVID-19 on Self-Reported Substa...
712304,PURPOSE OF REVIEW: COVID-19 pandemic has been ...,"In such circumstance, the CP therapy is undoub...",kf7lix7f,The immune plasma neutralizes the invading SAR...,[The concept of CP infusion evolved in 1880 wh...,document_parses/pdf_json/60c6c463404554c0ad087...,2021-07-03,COVID-19 Pandemic and the Convalescent Plasma ...
736570,OBJECTIVES: : To report the characteristics of...,NaN,e0eki2rk,Comparison between hesitant and pro-vaccinatio...,[The datasets generated and/or analyzed during...,document_parses/pdf_json/8ee6271860745bf4d23d6...,2022-03-28,{fr}Who is hesitant about Covid-19 vaccines ? ...
816108,Addressing COVID19 vaccine hesitancy and minim...,NaN,pyxmblll,NaN,[M a n u s c r i p t Abbreviations: PEG -polye...,document_parses/pdf_json/2aa4576b561138b59ca67...,2021-05-14,Immediate reactions following the first dose o...


Next, we can preprocess the text we got from the PDF JSON files. Preprocessing the text involves several tasks: first, we will convert all text to lowercase and remove stopwords. Next, we will perform lemmatization and store the preprocessed text in a separate column for further analysis.

In [14]:
stop_words = set(stopwords.words('english'))

# function that preprocesses the input string
def preprocess_text(string):
    string = string.lower()
    # remove stopwords
    string = " ".join([word for word in string.split()
                      if word not in stop_words])
    # lemmatization
    lemmatizer = nltk.stem.WordNetLemmatizer()
    string = " ".join([lemmatizer.lemmatize(word) for word in string.split()])
    return string

# function that preprocesses the text of a row in the sampled dataset
def preprocess_text_in_df(row):
    title = preprocess_text(row['title'])
    abstract = preprocess_text(row['abstract'])
    introduction = ""
    if pd.isnull(row['introduction']) == False:
        introduction = preprocess_text(row['introduction'])
    conclusion = ""
    if pd.isnull(row['conclusion']) == False:
        conclusion = preprocess_text(row['conclusion'])
    other_paragraph = ""
    for paragraph in row['other_paragraph']:
        other_paragraph += preprocess_text(paragraph)
    row["preprocessed_text"] = title + " " + abstract + " " + \
        introduction + " " + conclusion + " " + other_paragraph
    return row


tqdm.pandas(desc="preprocessing the text in the sampled dataset...")
df_sampled = df_sampled.progress_apply(
    lambda row: preprocess_text_in_df(row), axis=1)


preprocessing the text in the sampled dataset...: 100%|██████████| 10000/10000 [02:23<00:00, 69.47it/s]


In [15]:
df_sampled["preprocessed_text"].head()

753376    together: psychological wellbeing foreign work...
913530    impact covid-19 self-reported substance use, w...
712304    covid-19 pandemic convalescent plasma therapy:...
736570    {fr}who hesitant covid-19 vaccine ? profiling ...
816108    immediate reaction following first dose sars-c...
Name: preprocessed_text, dtype: object

### 2. Named Entity Recognition and Knowledge Base

#### 2.1 Entity extraction

To save time, we will only extract entities from the title, abstract, introduction, and conclusion sections of the text. These sections of the article are most likely to contain entities that are relevant to the topic of the article.

In [16]:
nlp = spacy.load("en_core_web_sm")

# function: get the name entities from the title, abstract, introduction, and conclusion of each data in the dataset
def get_name_entity(row):
    name_entity = []
    doc = nlp(row["title"])
    for ent in doc.ents:
        # if ent is not number
        if ent.label_ != "CARDINAL" and ent.label_ != "PERCENT" and ent.label_ != "MONEY":
            name_entity.append(ent.text)
    doc = nlp(row["abstract"])
    for ent in doc.ents:
        if ent.label_ != "CARDINAL" and ent.label_ != "PERCENT" and ent.label_ != "MONEY":
            name_entity.append(ent.text)
    if pd.isna(row["introduction"]) == False:
        doc = nlp(row["introduction"])
        for ent in doc.ents:
            if ent.label_ != "CARDINAL" and ent.label_ != "PERCENT" and ent.label_ != "MONEY":
                name_entity.append(ent.text)
    if pd.isna(row["conclusion"]) == False:
        doc = nlp(row["conclusion"])
        for ent in doc.ents:
            if ent.label_ != "CARDINAL" and ent.label_ != "PERCENT" and ent.label_ != "MONEY":
                name_entity.append(ent.text)
    return name_entity


In [17]:
# get the name entity of the title, abstract, introduction, and conclusion of each data in the dataset
tqdm.pandas(desc="Getting name entity...")
df_sampled['name_entity'] = df_sampled.progress_apply(
    lambda row: get_name_entity(row), axis=1)


Getting name entity...: 100%|██████████| 10000/10000 [12:24<00:00, 13.42it/s]


Presenting the frequency of the extracted entities.

In [18]:
# merge the name entity into one list
name_entity_list = []
for name_entity in df_sampled['name_entity']:
    name_entity_list.extend(name_entity)

# count the frequency of each name entity and sort it
name_entity_count = Counter(name_entity_list)
name_entity_count = sorted(name_entity_count.items(),
                           key=lambda x: x[1], reverse=True)

# show the top 100 name entity
name_entity_count[:100]


[('COVID-19', 34227),
 ('first', 3703),
 ('2019', 2925),
 ('China', 1873),
 ('2020', 1842),
 ('CI', 1834),
 ('second', 1709),
 ('COVID‐19', 1511),
 ('India', 1201),
 ('Wuhan', 919),
 ('daily', 875),
 ('Italy', 867),
 ('UK', 816),
 ('RNA', 794),
 ('IgG', 662),
 ('RBD', 654),
 ('al.', 644),
 ('Omicron', 637),
 ('the United States', 597),
 ('US', 591),
 ('2021', 578),
 ('third', 529),
 ('Delta', 527),
 ('Coronavirus', 423),
 ('ICU', 419),
 ('Italian', 406),
 ('Brazil', 398),
 ('Chinese', 394),
 ('Germany', 393),
 ('Spain', 377),
 ('WHO', 367),
 ('ACE2', 359),
 ('USA', 340),
 ('France', 339),
 ('U.S.', 336),
 ('COVID', 336),
 ('Pfizer', 333),
 ('December 2019', 331),
 ('Europe', 313),
 ('March 2020', 297),
 ('Moderna', 288),
 ('Vaccine', 287),
 ('PPE', 284),
 ('Japan', 284),
 ('European', 278),
 ('RT-PCR', 274),
 ('BCG', 272),
 ('England', 271),
 ('PCR', 270),
 ('Spike', 263),
 ('Australia', 261),
 ('S1', 256),
 ('Africa', 255),
 ('Saudi Arabia', 236),
 ('Israel', 232),
 ('Canada', 232),
 

#### 2.2 Knowledge base

We will manually build knowledge bases that display synonyms of the entities and their associated keywords. This will be based on the results of the Named Entity Recognition and the test query set.

In [21]:
# manually create the knowledge base

# create the knowledge base
knowledge_base_synonym = {
    "COVID-19": ["SARS-CoV-2", "coronavirus disease 2019", "coronavirus disease 19", "coronavirus 2019", "coronavirus 19", "2019 novel coronavirus", "2019-nCoV", "2019-novel coronavirus", "2019 novel coronavirus pneumonia", "2019-nCoV pneumonia"],
    "rapid testing": ["RAT", "rapid test", "rapid antigen test", "rapid antigen tests", "rapid antigen testing", "rapid antigen tests", "rapid antigen test kit"],
    "origin": ["origins", "source", "sources"],
    "initial symtoms": ["early signs"]
}

knowledge_base_association = {
    "mask": ["n95", "cloth mask"],
    "vaccine": ["mrna"],
    "origin": ["wuhan", "fish market"],
    "symptoms": ["fever", "chill", "cough", "tired", "headache", "loss taste or small", "sore throat", "diarrhea"],
    "sanitizer": ["alcohol"],
    "social distancing": ["quarantine", "lockdown"],
    "transmission route": ["airborne", "droplet", "contact", "fomite"],
    "testing": ["PCR"]
}


### 3. Indexing method

To efficiently retrieve documents containing words from the query, we will utilize the inverted index method. This method creates a dictionary that maps words to the documents that contain them.

In [22]:
# the unique word set of the preprocessed_text of the dataset
unique_word_set = set([])

def get_unique_words(row):
    for word in row['preprocessed_text'].split():
        # if not a single puctuation
        if re.match(r'^[^\w\s]$', word) == None:
            unique_word_set.add(word)


tqdm.pandas(desc="Getting unique words...")
result = df_sampled.progress_apply(lambda row: get_unique_words(row), axis=1)


Getting unique words...: 100%|██████████| 10000/10000 [00:36<00:00, 276.04it/s]


In [23]:
# the inverted index of the preprocessed_text of the dataset
inverted_index = inv_indx = {i: [] for i in unique_word_set}

# function: get the inverted index of the preprocessed_text of the dataset
def get_inverted_index(row, inverted_index):
    for word in row['preprocessed_text'].split():
        if word in inverted_index:
            inverted_index[word].append(row['cord_uid'])


tqdm.pandas(desc="Getting inverted index...")
result = df_sampled.progress_apply(
    lambda row: get_inverted_index(row, inverted_index), axis=1)


Getting inverted index...: 100%|██████████| 10000/10000 [02:06<00:00, 79.27it/s]


In [24]:
# get the documents containing the word "mask"
inverted_index["mask"]


['jon0u3zi',
 'jon0u3zi',
 'jon0u3zi',
 '7f6xlts3',
 '7f6xlts3',
 'x938j5rl',
 '8e8evwn4',
 '8e8evwn4',
 '1ykowy5k',
 '1ykowy5k',
 'trufildm',
 'trufildm',
 'trufildm',
 'trufildm',
 'trufildm',
 'trufildm',
 'trufildm',
 'trufildm',
 'u1qb4t25',
 'u1qb4t25',
 'u1qb4t25',
 'u1qb4t25',
 'u1qb4t25',
 'u1qb4t25',
 'u1qb4t25',
 'u1qb4t25',
 'u1qb4t25',
 'u1qb4t25',
 'u1qb4t25',
 'u1qb4t25',
 'u1qb4t25',
 'b9o6d24b',
 '75jxbqga',
 'yfwpd3pe',
 'mad91ezq',
 'mad91ezq',
 'mad91ezq',
 'mad91ezq',
 'igmhwm30',
 'igmhwm30',
 'gfqp659j',
 'b55op5ep',
 'b55op5ep',
 'b55op5ep',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',
 'eay6qfhz',

### 4. Text matching utility

To obtain the answer snippets to a query, we will first extend the query by adding related entities from the knowledge base. Next, we will use the inverted index method to locate documents containing the words from the extended query. We will then calculate cosine similarity between the query and the documents to rank them and select the top 5 as the target documents. Finally, we will again use cosine similarity to rank the paragraphs in the target documents and extract the top 3 paragraphs as answer snippets.

First, we define the function to preprocess the query.

In [25]:
# function: extend the query with the knowledge base
def extend_query(query, knowledge_base_synonym, knowledge_base_association):
    # convert the query to lower case
    query = query.lower()
    # for each key in the knowledge base, if the key is in the query, then add the value to the query string
    for key in knowledge_base_synonym:
        # if the query string contains the key
        if key.lower() in query:
            # concatenate the query string with each value of the key
            for value in knowledge_base_synonym[key]:
                query += " " + value
    for key in knowledge_base_association:
        if key.lower() in query:
            for value in knowledge_base_association[key]:
                query += " " + value
    return query

# query preprocessing: extend the query with the knowledge base, remove stopwords, lemmatization, and remove question mark
def preprocess_query(query):
    # extend the query with the knowledge base
    query = extend_query(query, knowledge_base_synonym,
                         knowledge_base_association)
    query = query.lower()
    # remove stopwords
    query = " ".join([word for word in query.split()
                     if word not in stop_words])
    # lemmatization
    lemmatizer = nltk.stem.WordNetLemmatizer()
    query = " ".join([lemmatizer.lemmatize(word) for word in query.split()])
    # remove question mark
    query = query.replace("?", "")
    return query


Next, we need to define the function to embed the query and the documents.

In [26]:
# function: get the vector representation of the query by averaging the vector representation of each word in the query
def embedding_string(string):
    vectors = []
    for word in string.split():
        if word in glove_model:
            vectors.append(glove_model[word])
    if vectors == []:
        return np.zeros(100)
    return np.mean(vectors, axis=0)

# get the vector representation of each document in df_sampled
df_sampled['embedding'] = df_sampled.progress_apply(
    lambda row: embedding_string(row['preprocessed_text']), axis=1)

df_sampled['embedding']

Getting inverted index...: 100%|██████████| 10000/10000 [01:28<00:00, 113.42it/s]


753376     [-0.062524386, 0.20725419, 0.11635797, 0.14362...
913530     [-0.08254339, 0.176392, 0.11337998, 0.13646367...
712304     [-0.0043793595, 0.18165466, -0.0017466267, 0.1...
736570     [-0.045271114, 0.20421785, 0.08802674, 0.15633...
816108     [-0.06078614, 0.21743849, 0.14221364, 0.168882...
                                 ...                        
768690     [-0.00047997944, 0.033717453, 0.10655, -0.0020...
737001     [-0.03618455, 0.11873989, 0.016081613, 0.05789...
1004462    [-0.06748906, 0.27562916, 0.14938413, 0.153400...
79013      [-0.13417636, -0.071013786, -0.23611803, -0.13...
25116      [-0.002016765, 0.18712643, 0.1122159, 0.080348...
Name: embedding, Length: 10000, dtype: object

#### 4.1 Cosine similarity

In [27]:
# function: calculate the cosine similarity between two vectors
def cosine_similarity(vector1, vector2):
    return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

#### 4.2 Get the top n documents or paragraphs

In [28]:
# function: get the top n documents that are most similar to the query from the indexed document set
def get_top_n_documents(query_vector, indexed_document_set, n):
    # get the vector representation of each document in the indexed document set
    df_indexed = df_sampled[df_sampled['cord_uid'].isin(indexed_document_set)]
    # calculate the cosine similarity between the query and each document in the indexed document set
    df_indexed['similarity'] = df_indexed.progress_apply(
        lambda row: cosine_similarity(query_vector, row['embedding']), axis=1)
    # sort the documents by the similarity score
    df_indexed = df_indexed.sort_values(by=['similarity'], ascending=False)
    # get the top n documents
    df_top_n = df_indexed[:n]
    return df_top_n


In [29]:
# function: get the top n paragraphs that are most similar to the query for each document in the top n documents
def get_top_n_paragraphs(query_vector, df_top_n_doc, n):
    top_n_paragraphs_dict = {}
    for index, row in df_top_n_doc.iterrows():
        # get the paragraphs of the article
        paragraph_list = {"paragraph": [], "similarity": []}
        paragraph_list["paragraph"].append(row["abstract"])

        # if the conclusion is not null, then add it to the paragraph list
        if pd.isnull(row["conclusion"]) == False:
            paragraph_list["paragraph"].append(row["conclusion"])

        if pd.isnull(row["introduction"]) == False:
            paragraph_list["paragraph"].append(row["introduction"])

        paragraph_list["paragraph"].extend(row["other_paragraph"])

        # calculate the cosine similarity between the query and each paragraph
        for paragraph in paragraph_list["paragraph"]:
            paragraph_list["similarity"].append(cosine_similarity(
                query_vector, embedding_string(paragraph)))
        
        df_paragraph_list = pd.DataFrame(paragraph_list)
        # sort the paragraphs by the similarity score
        df_paragraph_list = df_paragraph_list.sort_values(
            by=['similarity'], ascending=False)
        top_n_paragraphs_dict[row["cord_uid"]] = df_paragraph_list[:n]
    return top_n_paragraphs_dict


#### 4.3 Get the final answer of a query

In [31]:
# function: integrate the above functions to get the answer of the query
def get_answer(query, n):
    # preprocess the query
    query = preprocess_query(query)
    # get the indexed document set
    indexed_document_set = set([])
    for word in query.split():
        if word in inverted_index:
            for document in inverted_index[word]:
                indexed_document_set.add(document)
    # get the vector representation of the query
    query_vector = embedding_string(query)
    # get the top n documents that are most similar to the query
    df_top_n = get_top_n_documents(query_vector, indexed_document_set, n)
    # get the top n paragraphs that are most similar to the query for each document in the top n documents
    top_n_paragraphs_dict = get_top_n_paragraphs(query_vector, df_top_n, 3)
    return top_n_paragraphs_dict


### 5. Test utility and test results

In [33]:
# the test set with 10 queries
test_set = [
    "what is the origin of COVID-19?",
    "what types of rapid testing for Covid-19 have been developed?",
    "has social distancing had an impact on slowing the spread of COVID-19?",
    "what are the transmission routes of coronavirus?",
    "what are the best masks for preventing infection by Covid-19?",
    "what type of hand sanitizer is needed to destroy Covid-19?",
    "What vaccine candidates are being tested for Covid-19?",
    "does Vitamin D impact COVID-19 prevention and treatment?",
    "how long can the coronavirus live outside the body?",
    "what are the initial symptoms of Covid-19?",
]


In [34]:
# present the top_n_paragraphs_dict in a more readable way
def present_answer(top_n_paragraphs_dict):
    for key in top_n_paragraphs_dict:
        # print the document title
        print("Document: " +
              df_sampled[df_sampled["cord_uid"] == key]["title"].values[0])
        for index, row in top_n_paragraphs_dict[key].iterrows():
            print("Paragraph: " + row["paragraph"])
            print("Confidence: " + str(row["similarity"]))
            print("")
        print("---------------------------------------------------------------------------------------------")


In [35]:
for i in range(len(test_set)):
    print("Question " + str(i + 1) + ": " + test_set[i])
    present_answer(get_answer(test_set[i], 5))
    print("")
    print("=================================================================================================")
    print("=================================================================================================")
    print("")
    print("")


Question 1: what is the origin of COVID-19?


Getting inverted index...: 100%|██████████| 9990/9990 [00:00<00:00, 32431.44it/s]


Document: Immune response in COVID-19: What do we currently know?
Paragraph: The acute respiratory syndrome is a disease caused by the SARS-CoV-2 virus (COVID-37 19), where symptoms include difficulty breathing, high fever, and cough (WHO, 2020). 
Confidence: 0.6983451843261719

Paragraph: In 2002/2003 there was a pandemic denominate SARS (severe acute respiratory syndrome), caused by the SARS-CoV virus that belongs to the genera Betacoranavirus and the family Coronaviridae, generally responsible for influenza infections. In mid of 2019, a new disease by the coronavirus named by COVID-19 (SARS-CoV-2) emerged, both infections have flu symptoms, however they are infections that variable intensity, being medium to severe. In medium infections individuals have the virus and exhibit symptoms, however hospitalization is not necessary, in severe infections, individuals are hospitalized, have high pathology and in some cases progress to death. The virus is formed by simple positive RNA, envelo

Getting inverted index...: 100%|██████████| 9994/9994 [00:00<00:00, 34491.54it/s]


Document: Reduced neutralization of SARS-CoV-2 B.1.617 variant by convalescent and vaccinated sera
Paragraph: Coronavirus disease 2019 (COVID-19) is caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The Spike protein that mediates coronavirus entry into host cells is a major target for COVID-19 vaccines and antibody therapeutics. However, multiple variants of SARS-CoV-2 have emerged, which may potentially compromise vaccine effectiveness. Using a pseudovirus-based assay, we evaluated SARS-CoV-2 cell entry mediated by the viral Spike B.1.617 and B.1.1.7 variants. We also compared the neutralization ability of monoclonal antibodies from convalescent sera and neutralizing antibodies (NAbs) elicited by CoronaVac (inactivated vaccine) and ZF2001 (RBD-subunit vaccine) against B.1.617 and B.1.1.7 variants. Our results showed that, compared to D614G and B.1.1.7 variants, B.1.617 shows enhanced viral entry and membrane fusion, as well as more resistant to antibody neutrali

...

Document: Immune response in COVID-19: What do we currently know?
Paragraph: The acute respiratory syndrome is a disease caused by the SARS-CoV-2 virus (COVID-37 19), where symptoms include difficulty breathing, high fever, and cough (WHO, 2020). 
Confidence: 0.6615413427352905

Paragraph: In 2002/2003 there was a pandemic denominate SARS (severe acute respiratory syndrome), caused by the SARS-CoV virus that belongs to the genera Betacoranavirus and the family Coronaviridae, generally responsible for influenza infections. In mid of 2019, a new disease by the coronavirus named by COVID-19 (SARS-CoV-2) emerged, both infections have flu symptoms, however they are infections that variable intensity, being medium to severe. In medium infections individuals have the virus and exhibit symptoms, however hospitalization is not necessary, in severe infections, individuals are hospitalized, have high pathology and in some cases progress to death. The virus is formed by simple positive RNA, envelo

Getting inverted index...: 100%|██████████| 8227/8227 [00:00<00:00, 31932.97it/s]


Document: Stitches in the Heart: A Small History of Scars
Paragraph: The human heart consists of knotted tubes through which the living flows are propelled onwards. It beats silently in the dark chamber of the chest. Oxygenised blood, coming from the lungs, is transported through arteries and veins mostly unperceived. A mode of disappearance of our inner rhythmic alternation is at work, a central absence of the visceral processes. Inspired by philosopher Drew Leder's study The Absent Body (1990) , 'I receive the surrounding world through my eyes, my ears, my hands. The structure of my perceptual organs shapes that which I apprehend' (1). I will introduce a concept in which the heart as carnal material of modification and as delicate fabric from which love is made are inextricably intertwined with each other.
Confidence: 0.8525002002716064

Paragraph: Nevertheless, I took my 16 mm vintage camera for a walk a few days later. I found Vienna's heart at the first district deeply wounded. Bu

Getting inverted index...: 100%|██████████| 9995/9995 [00:00<00:00, 34573.73it/s]


Document: Immune response in COVID-19: What do we currently know?
Paragraph: The acute respiratory syndrome is a disease caused by the SARS-CoV-2 virus (COVID-37 19), where symptoms include difficulty breathing, high fever, and cough (WHO, 2020). 
Confidence: 0.7719785571098328

Paragraph: In 2002/2003 there was a pandemic denominate SARS (severe acute respiratory syndrome), caused by the SARS-CoV virus that belongs to the genera Betacoranavirus and the family Coronaviridae, generally responsible for influenza infections. In mid of 2019, a new disease by the coronavirus named by COVID-19 (SARS-CoV-2) emerged, both infections have flu symptoms, however they are infections that variable intensity, being medium to severe. In medium infections individuals have the virus and exhibit symptoms, however hospitalization is not necessary, in severe infections, individuals are hospitalized, have high pathology and in some cases progress to death. The virus is formed by simple positive RNA, envelo

#### 5.1 Calculate the MRR of the result

After obtaining the test results, we will manually identify the correctness of each return snippets and calculate the reciprocal rank for each query.

In [36]:
reciprocal_rank = np.zeros(len(test_set))
reciprocal_rank[0] = 1/2 # the second document is a relevant document
reciprocal_rank[1] = 1/5 # the fifth document is a relevant document
reciprocal_rank[2] = 0 # there is no relevant document in the top 5 documents
reciprocal_rank[3] = 1 # the first document is a relevant document
reciprocal_rank[4] = 0
reciprocal_rank[5] = 0
reciprocal_rank[6] = 1/4
reciprocal_rank[7] = 0
reciprocal_rank[8] = 0
reciprocal_rank[9] = 1

print("The mean reciprocal rank of the test set is: " +
      str(np.mean(reciprocal_rank)))


The mean reciprocal rank of the test set is: 0.29500000000000004


#### 5.2 Result analysis

The result shows that only half of the queries are accurately answered, which are: 1. "what is the origin of COVID-19?"; 2. "what types of rapid testing for Covid-19 have been developed?"; 3. "what are the transmission routes of coronavirus?"; 4. "What vaccine candidates are being tested for Covid-19?"; 5. "what are the initial symptoms of Covid-19?". The success of these queries can be attributed to the presence of specific keywords in the knowledge base. Conversely, queries that were answered incorrectly lacked keywords that were not present in the knowledge base or dataset. Therefore, it can be concluded that a well-constructed knowledge base is critical for precise answer retrieval, which can be enhanced by incorporating additional keywords and synonyms or by utilizing relation extraction to establish the connections between entities. Despite these limitations, the outcomes highlight the potential of utilizing named entity recognition and knowledge bases for question-answering systems.

### 6. Simple user interface

In [93]:
convert_json_list = []
def convert_to_json(row):
    num = 0
    if(row["cord_uid"] == "rmvw3ko0"):
        print(row)
    for i in row['other_paragraph']:
         
        if(pd.isnull(i) == False):
            convert_json_list.append(
            {
                "id": row["cord_uid"]+"-"+ str(num),
                "contents": i
            })
            num = num+1;
        

In [94]:
!rm -r collection.json
x = df_sampled.progress_apply(
    lambda row: convert_to_json(row), axis=1)

json_str = json.dumps(convert_json_list)

with open("collection.json", "w") as outfile:
    outfile.write(json_str)
print(json_str[0:500])
print(convert_json_list[0:100])

rm: cannot remove 'collection.json': No such file or directory


Getting inverted index...:   0%|          | 0/10000 [00:00<?, ?it/s]

abstract             As the COVID-19 pandemic sweeps the globe, man...
conclusion                                                         NaN
cord_uid                                                      rmvw3ko0
introduction                                                       NaN
other_paragraph      [Since March 2020, the SARS-CoV-2 (COVID-19) p...
pdf_json_files       document_parses/pdf_json/761ee085a77ce60863fe9...
publish_time                                                2022-04-11
title                The Impact of COVID-19 on Self-Reported Substa...
preprocessed_text    impact covid-19 self-reported substance use, w...
name_entity          [Functioning Among, United States, Veterans, C...
embedding            [-0.08254339, 0.176392, 0.11337998, 0.13646367...
Name: 913530, dtype: object


Getting inverted index...: 100%|██████████| 10000/10000 [00:02<00:00, 3598.66it/s]


[{"id": "0ex7dy6g-0", "contents": "A severe acute respiratory syndrome, coronavirus 2 (SARS-CoV-2), later known as COVID-19, was identified in the Hubei Province of China towards the end of 2019. By 30 April 2020, the virus had spread to more than 200 different countries causing 217, 769 deaths worldwide, with the per-capita mortality and infection rates varying greatly between nations (World Health Organization, 2020). Part of the variance in infection and mortality rates can be attributed the 
[{'id': '0ex7dy6g-0', 'contents': 'A severe acute respiratory syndrome, coronavirus 2 (SARS-CoV-2), later known as COVID-19, was identified in the Hubei Province of China towards the end of 2019. By 30 April 2020, the virus had spread to more than 200 different countries causing 217, 769 deaths worldwide, with the per-capita mortality and infection rates varying greatly between nations (World Health Organization, 2020). Part of the variance in infection and mortality rates can be attributed the

In [95]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input ./ \
  --index index \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

2023-04-05 07:21:23,634 INFO  [main] index.IndexCollection (IndexCollection.java:250) - Setting log level to INFO
2023-04-05 07:21:23,637 INFO  [main] index.IndexCollection (IndexCollection.java:253) - Starting indexer...
2023-04-05 07:21:23,638 INFO  [main] index.IndexCollection (IndexCollection.java:254) - ============ Loading Parameters ============
2023-04-05 07:21:23,639 INFO  [main] index.IndexCollection (IndexCollection.java:255) - DocumentCollection path: ./
2023-04-05 07:21:23,639 INFO  [main] index.IndexCollection (IndexCollection.java:256) - CollectionClass: JsonCollection
2023-04-05 07:21:23,640 INFO  [main] index.IndexCollection (IndexCollection.java:257) - Generator: DefaultLuceneDocumentGenerator
2023-04-05 07:21:23,641 INFO  [main] index.IndexCollection (IndexCollection.java:258) - Threads: 1
2023-04-05 07:21:23,642 INFO  [main] index.IndexCollection (IndexCollection.java:259) - Language: en
2023-04-05 07:21:23,645 INFO  [main] index.IndexCollection (IndexCollection.jav

In [131]:
from pyserini.search.lucene import LuceneSearcher
import json
searcher = LuceneSearcher('./index')
f = open("collection.json", "r")
text = json.loads(f.read())


hits = searcher.search('which animal did corona spread from')

for i in range(len(hits)):
   
    print(f'{i+1:2} {hits[i].docid} {hits[i].contents} {hits[i].score:.5f}')
    for j in text:
        
        if(j['id'] == hits[i].docid):
            print(j['contents'])

 1 wihmbqam-1 None 8.52080
Coronavirus disease 2019 (COVID-19) is a virus of the Corona virus family and the source of a respiratory illness outbreak throughout the world that originated in Wuhan, China. Studies (1) (2) (3) show that Covid-19 has clinical characteristics akin to the SARS-CoV. The dominant symptoms include fever and cough, while gastrointestinal symptoms are uncommon. In COVID-19 infected patients the absence of fever is more frequent than in patients infected by similar viruses, i.e., MERS Corona Virus (2%) and SARS Corona Virus (1%) (4); therefore, there is a possibility of non-febrile patients being missed by a surveillance mechanism with a primary focus on detecting fever (5) . The initial patients infected by COVID-19, reportedly indicated an association with a large seafood and animal market in Wuhan that demonstrated an animal-to-person spread. Per contra, a burgeoning number of patients have not displayed any association with the animal markets, revealing the fa

In [ ]:
test_summerized_reference = "We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models (Peters et al., 2018a; Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial taskspecific architecture modifications. BERT is conceptually simple and empirically powerful. It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE score to 80.5% (7.7% point absolute improvement), MultiNLI accuracy to 86.7% (4.6% absolute improvement), SQuAD v1.1 question answering Test F1 to 93.2 (1.5 point absolute improvement) and SQuAD v2.0 Test F1 to 83.1 (5.1 point absolute improvement)."
test_query = "What does the 'B' in BERT stand for?"

In [134]:
covid_text = "Origin In December 2019, the reports of patients with viral pneumonia in China surfaced. The origin was related to the Huanan Seafood's whole market. Researchers established that the disease has most likely originated from bats, mutated to infect humans, and transmitted by droplet routes among humans. Since the origin occurred in a wet market, the media criticized the Chinese for consuming live animals like bats, snakes, and dogs. Previously known as \"Wuhan virus\" and \"China coronavirus,\" it was subsequently called 2019-nCoV, and then finally, on February 11, the World Health Organization (WHO) gave the disease an official name, \"Covid-19\". In spite of this, renowned world leaders ignorantly kept calling it \"Chinese virus\" which left concerning effects in international communities [8, 9] . Asian people were stigmatized and called by the name 'Corona' on the streets. Many reported incidents of such racial slurs which exaggerated already existing prejudices among people. This happened in SARS 2003 in Toronto which caused the xenophobic reaction, and similarly, was repeated with the coronavirus pandemic. It had a significant impact, leading the Chinese government to order the closure of all farmer's markets and put a ban on eating live animals. This was an essential step in halting the spread of the virus. However, it also created a false sense of security among the rest of the world, and no one took the necessary precautions when the cases were limited to China"
covid_question = "which animal did corona spread from"

In [98]:
!pip install transformers
import torch

In [101]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [103]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [105]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
#     print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example through the model.
    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                    token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                    return_dict=True) 

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')


In [135]:
answer_question(covid_question,covid_text )

Answer: "bats"


In [ ]:
# accept the query from the user
query = input("Please enter your question: ")
print(query)
# get the answer of the query
top_n_paragraphs_dict = get_answer(query, 5)
# present the answer
present_answer(top_n_paragraphs_dict)


### 7. References